In [1]:
"""Cash ladder

Demonstration of how to compute a cash ladder for a portfolio.

Attributes
----------
cash
"""

'Cash ladder\n\nDemonstration of how to compute a cash ladder for a portfolio.\n\nAttributes\n----------\ncash\n'

# Cash Ladder

In this notebook, we demonstrate how users can create a cash ladder with LUSID. For the purposes of this notebook, we will set upsert a portfolio and some example transactions. Then we will use bucketed cashflows to understand how those transactions have affected our balance and create a cash ladder.

## Setup LUSID

We start by importing relevant libraries, authenticating our user, and creating our API client.

In [18]:
# Import general purpose packages
import os
import json
import pandas as pd
import numpy as np
import pytz
import warnings
from datetime import datetime, timedelta
# from cashladder import get_cash_ladder
# import ipywidgets as widgets
from IPython.display import display#, Markdown, clear_output

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Configure notebook warnings and pandas display
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

# Define the relevant APIs we will use
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)

## 1) Prepare Setup Data 

In this notebook, we have a portfolio called EQUITY_FUND.

In [19]:
# Load a mapping file for loading data
with open("config/cash_ladder_seed_data.json", "r") as seed_data_config:
    mapping_file = json.loads(seed_data_config.read())
    
# Portfolio scope and code
scope = "cash-ladder-001"
portfolio_code = "EQUITY_FUND"

# Load a file of equity transactions
transactions_file = r"data/cash_ladder.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code

# Load portfolios, instruments, and transactions
seed_data_response = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_file,
    "csv",
    mappings = mapping_file
)

## 2) Upload Quote Data

We will load in some quotes for our equity instruments.

In [20]:
equity_prices = pd.read_csv("data/equity_quotes.csv")
equity_prices.head(n=5)

,instrument_id,instrument_name,date,price,currency
0,EQ_GB001,vodafone,29/03/2020,114.6,GBP
1,EQ_GB001,vodafone,30/03/2020,114.5,GBP
2,EQ_GB001,vodafone,31/03/2020,118.54,GBP
3,EQ_GB001,vodafone,01/04/2020,117.64,GBP
4,EQ_GB001,vodafone,02/04/2020,117.46,GBP


We will then upsert these quotes to LUSID.

In [24]:
# Iterate through our dataframe and add each quote to an UpsertQuoteRequest
instrument_quotes = {
    index: models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider="Lusid",
                instrument_id=row["instrument_id"],
                instrument_id_type="ClientInternal",
                quote_type="Price",
                field="mid",
            ),
            effective_at=str(datetime.strptime(row['date'], '%d/%m/%Y').date()),
        ),
        metric_value=models.MetricValue(value=row['price'], unit=row['currency']),
        scale_factor=100,
    )
    for index, row in equity_prices.iterrows()
}

# Upser the quotes
response = quotes_api.upsert_quotes(scope=scope, request_body=instrument_quotes)

if response.failed == {}:
    print(f"Quotes successfully loaded into LUSID. {len(response.values)} quotes loaded.")
else:
    print(f"Some failures occurred during quote upsertion, {len(response.failed)} did not get loaded into LUSID.")

Quotes successfully loaded into LUSID. 39 quotes loaded.


## 3) Create configuration recipe

Create and upsert basic configuration recipe to be used for valuation.

In [21]:
# Create a configuration recipe
configuration_recipe = models.ConfigurationRecipe(
    scope=scope,
    code=portfolio_code,
    
    # Define the market rules and which quotes to use
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Quote.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D.0D",
            )
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        )
    )
)

# Upsert the recipe to LUSID
config_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

## Cash ladder via `GetPortfolioCashLadder`

In [25]:
response = transaction_portfolio_api.get_portfolio_cash_ladder(
        scope=scope, 
        code=portfolio_code, 
        from_effective_at=datetime(year=2020, month=3, day=25, tzinfo=pytz.UTC),
        to_effective_at=datetime(year=2020, month=4, day=6, tzinfo=pytz.UTC),
        effective_at=datetime(year=2020, month=4, day=1, tzinfo=pytz.UTC),
        recipe_id_scope=scope,
        recipe_id_code=portfolio_code,
    )

for value in response.values:
    ladder = pd.DataFrame(columns=["ReportRunTime", "SettlementDate", "Currency", "Portfolio", "CashLadderStatus", "CashImpact"])
    for record in value.records:
        ladder.loc[len(ladder)] = {
            "ReportRunTime": "11:25:21",
            "SettlementDate": record.effective_date,
            "Currency": value.currency,
            "Portfolio": portfolio_code,
            "CashLadderStatus": "SOD Balance",
            "CashImpact": record.open
        }
        
        for item in record.activities:
            ladder.loc[len(ladder)] = {
                "ReportRunTime": "11:25:21",
                "SettlementDate": record.effective_date,
                "Currency": value.currency,
                "Portfolio": portfolio_code,
                "CashLadderStatus": f"Cash impact from a {item} transaction",
                "CashImpact": record.activities[item]
            }
        
        ladder.loc[len(ladder)] = {
            "ReportRunTime": "11:25:21",
            "SettlementDate": record.effective_date,
            "Currency": value.currency,
            "Portfolio": portfolio_code,
            "CashLadderStatus": "EOD Balance",
            "CashImpact": record.close
        }

    display(ladder)

,ReportRunTime,SettlementDate,Currency,Portfolio,CashLadderStatus,CashImpact
0,11:25:21,2020-03-30 00:00:00+00:00,USD,EQUITY_FUND,SOD Balance,0.0
1,11:25:21,2020-03-30 00:00:00+00:00,USD,EQUITY_FUND,Cash impact from a Side1 transaction,20000000.0
2,11:25:21,2020-03-30 00:00:00+00:00,USD,EQUITY_FUND,EOD Balance,20000000.0
3,11:25:21,2020-04-01 00:00:00+00:00,USD,EQUITY_FUND,SOD Balance,20000000.0
4,11:25:21,2020-04-01 00:00:00+00:00,USD,EQUITY_FUND,Cash impact from a Side2 transaction,-3000000.0
5,11:25:21,2020-04-01 00:00:00+00:00,USD,EQUITY_FUND,EOD Balance,17000000.0
6,11:25:21,2020-04-03 00:00:00+00:00,USD,EQUITY_FUND,SOD Balance,17000000.0
7,11:25:21,2020-04-03 00:00:00+00:00,USD,EQUITY_FUND,Cash impact from a Side2 transaction,0.0
8,11:25:21,2020-04-03 00:00:00+00:00,USD,EQUITY_FUND,Cash impact from a Side1 transaction,-2000000.0
9,11:25:21,2020-04-03 00:00:00+00:00,USD,EQUITY_FUND,EOD Balance,15000000.0


,ReportRunTime,SettlementDate,Currency,Portfolio,CashLadderStatus,CashImpact
0,11:25:21,2020-03-31 00:00:00+00:00,GBP,EQUITY_FUND,SOD Balance,0.0
1,11:25:21,2020-03-31 00:00:00+00:00,GBP,EQUITY_FUND,Cash impact from a Side1 transaction,500000.0
2,11:25:21,2020-03-31 00:00:00+00:00,GBP,EQUITY_FUND,EOD Balance,500000.0
3,11:25:21,2020-04-01 00:00:00+00:00,GBP,EQUITY_FUND,SOD Balance,500000.0
4,11:25:21,2020-04-01 00:00:00+00:00,GBP,EQUITY_FUND,Cash impact from a Side2 transaction,-200000.0
5,11:25:21,2020-04-01 00:00:00+00:00,GBP,EQUITY_FUND,Cash impact from a Side1 transaction,10000000.0
6,11:25:21,2020-04-01 00:00:00+00:00,GBP,EQUITY_FUND,EOD Balance,10300000.0
7,11:25:21,2020-04-03 00:00:00+00:00,GBP,EQUITY_FUND,SOD Balance,10300000.0
8,11:25:21,2020-04-03 00:00:00+00:00,GBP,EQUITY_FUND,Cash impact from a Side2 transaction,-6900000.0
9,11:25:21,2020-04-03 00:00:00+00:00,GBP,EQUITY_FUND,Cash impact from a Side1 transaction,-1000000.0


## 4) Produce Cash Ladder

## Cash ladder via `GetBucketedCashFlows`

In [26]:
bucket_response = transaction_portfolio_api.get_bucketed_cash_flows(
    scope=scope,
    code=portfolio_code,
    bucketed_cash_flow_request=models.BucketedCashFlowRequest(
        rounding_method="RoundUp",
        bucketing_dates=[
            datetime(year=2020, month=3, day=30, tzinfo=pytz.UTC),
            datetime(year=2020, month=3, day=31, tzinfo=pytz.UTC),
            datetime(year=2020, month=4, day=1, tzinfo=pytz.UTC),
            datetime(year=2020, month=4, day=2, tzinfo=pytz.UTC),
            datetime(year=2020, month=4, day=3, tzinfo=pytz.UTC),
            datetime(year=2020, month=4, day=4, tzinfo=pytz.UTC),
            datetime(year=2020, month=4, day=5, tzinfo=pytz.UTC),
            datetime(year=2020, month=4, day=6, tzinfo=pytz.UTC),
        ],
        recipe_id=models.ResourceId(
            scope=scope,
            code=portfolio_code,
        ),
        effective_at=datetime(year=2020, month=4, day=1, tzinfo=pytz.UTC),
        cash_flow_type="PortfolioCashFlow"
    )
)

# print(bucket_response.data)
ladder = pd.DataFrame(columns=["ReportRunTime", "SettlementDate", "Currency", "Portfolio", "CashLadderStatus", "CashImpact"])
for bucket in bucket_response.data:
    ladder.loc[len(ladder)] = {
        "ReportRunTime": "11:25:21",
        "SettlementDate": bucket['Valuation/CashFlowDate/RoundUp'],
        "Currency": bucket['Valuation/CashFlowCurrency'],
        "Portfolio": portfolio_code,
        "CashLadderStatus": "SOD Balance",
        "CashImpact": bucket['SumCumulativeInArrears(Valuation/CashFlowAmount)']
    }
    
    for cashflow in bucket['Append(Valuation/CashFlowAmount)']:
        if int(cashflow['value']) == 0:
            continue
        else:
            ladder.loc[len(ladder)] = {
                "ReportRunTime": "11:25:21",
                "SettlementDate": bucket['Valuation/CashFlowDate/RoundUp'],
                "Currency": bucket['Valuation/CashFlowCurrency'],
                "Portfolio": portfolio_code,
                "CashLadderStatus": "Cash impact from a transaction",
                "CashImpact": cashflow['value'],
            }

    ladder.loc[len(ladder)] = {
        "ReportRunTime": "11:25:21",
        "SettlementDate": bucket['Valuation/CashFlowDate/RoundUp'],
        "Currency": bucket['Valuation/CashFlowCurrency'],
        "Portfolio": portfolio_code,
        "CashLadderStatus": "EOD Balance",
        "CashImpact": bucket['SumCumulativeInAdvance(Valuation/CashFlowAmount)']
    }
    
currencies = ladder.Currency.unique()
curr_dict = {curr : pd.DataFrame() for curr in currencies}
for curr in curr_dict.keys():
    curr_dict[curr] = ladder[:][ladder.Currency == curr]
    
display(curr_dict['USD'])
display(curr_dict['GBP'])

,ReportRunTime,SettlementDate,Currency,Portfolio,CashLadderStatus,CashImpact
0,11:25:21,2020-03-30T00:00:00.0000000+00:00,USD,EQUITY_FUND,SOD Balance,NaN
1,11:25:21,2020-03-30T00:00:00.0000000+00:00,USD,EQUITY_FUND,Cash impact from a transaction,20000000.0
2,11:25:21,2020-03-30T00:00:00.0000000+00:00,USD,EQUITY_FUND,EOD Balance,20000000.0
3,11:25:21,2020-04-01T00:00:00.0000000+00:00,USD,EQUITY_FUND,SOD Balance,20000000.0
4,11:25:21,2020-04-01T00:00:00.0000000+00:00,USD,EQUITY_FUND,Cash impact from a transaction,-3000000.0
5,11:25:21,2020-04-01T00:00:00.0000000+00:00,USD,EQUITY_FUND,EOD Balance,17000000.0
6,11:25:21,2020-04-03T00:00:00.0000000+00:00,USD,EQUITY_FUND,SOD Balance,17000000.0
7,11:25:21,2020-04-03T00:00:00.0000000+00:00,USD,EQUITY_FUND,Cash impact from a transaction,-3000000.0
8,11:25:21,2020-04-03T00:00:00.0000000+00:00,USD,EQUITY_FUND,Cash impact from a transaction,3000000.0
9,11:25:21,2020-04-03T00:00:00.0000000+00:00,USD,EQUITY_FUND,Cash impact from a transaction,-2000000.0


,ReportRunTime,SettlementDate,Currency,Portfolio,CashLadderStatus,CashImpact
16,11:25:21,2020-03-30T00:00:00.0000000+00:00,GBP,EQUITY_FUND,SOD Balance,NaN
17,11:25:21,2020-03-30T00:00:00.0000000+00:00,GBP,EQUITY_FUND,EOD Balance,0.0
18,11:25:21,2020-03-31T00:00:00.0000000+00:00,GBP,EQUITY_FUND,SOD Balance,0.0
19,11:25:21,2020-03-31T00:00:00.0000000+00:00,GBP,EQUITY_FUND,Cash impact from a transaction,500000.0
20,11:25:21,2020-03-31T00:00:00.0000000+00:00,GBP,EQUITY_FUND,EOD Balance,500000.0
21,11:25:21,2020-04-01T00:00:00.0000000+00:00,GBP,EQUITY_FUND,SOD Balance,500000.0
22,11:25:21,2020-04-01T00:00:00.0000000+00:00,GBP,EQUITY_FUND,Cash impact from a transaction,-200000.0
23,11:25:21,2020-04-01T00:00:00.0000000+00:00,GBP,EQUITY_FUND,Cash impact from a transaction,10000000.0
24,11:25:21,2020-04-01T00:00:00.0000000+00:00,GBP,EQUITY_FUND,EOD Balance,10300000.0
25,11:25:21,2020-04-03T00:00:00.0000000+00:00,GBP,EQUITY_FUND,SOD Balance,10300000.0
